In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import scipy as sp
import scipy.stats
import pandas as pd
import numpy as np
from tqdm import tqdm
import itertools
import re
res_digit = r'[0-9]'

# fourier transform
from scipy.fft import fft, ifft

from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.manifold import TSNE



In [2]:
# This is a hack to make the library in the parent folder available for imoprts
# A better solution is by np8 here:
# https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder
import sys
import os
import inspect

thisdir = sys.path[0]
print(f"thisdir = {thisdir}")
parentdir = os.path.dirname(thisdir)
#print(f"parentdir = {parentdir}")
if not parentdir in sys.path:
    print("Adding parent directory to python path")
    sys.path.insert(1, parentdir)
else:
    print("Skipping adding parent direct to path (there already)")

print(f"sys.path =\n{sys.path}")



thisdir = /home/luke/git/external/predicament/notebooks
Adding parent directory to python path
sys.path =
['/home/luke/git/external/predicament/notebooks', '/home/luke/git/external/predicament', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/luke/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/lib/python3.10/dist-packages']


In [3]:
## ensure relative path to data directory is sound
# for the notebook we need to modify the BASE_DATA_FOLDER
import os 
os.environ['PREDICAMENT_DATA_DIR'] =  '../data'

from predicament.utils.config import DREEM_EEG_CHANNELS
from predicament.utils.config import RESULTS_BASE_PATH

In [4]:
from predicament.data.timeseries import create_participant_data_edf_only
from predicament.data.windowed import window_all_participants_data
from predicament.data.windowed import merge_condition_data
from predicament.data.partitioning import between_subject_cv_partition

from predicament.data.features import MAXIMAL_FEATURE_GROUP
from predicament.data.features import STATS_FEATURE_GROUP
from predicament.data.features import INFO_FEATURE_GROUP
from predicament.data.features import FREQ_FEATURE_GROUP
from predicament.data.features import convert_timeseries_to_features
from prepare_evaluation_data import load_dataframe_and_config


## Balanced data, held out participants

In [6]:
res_fname = 'balanced_eeg_2023-12-12T08:43:49.csv'
import os
res_fpath = os.path.join(RESULTS_BASE_PATH, res_fname)
df = pd.read_csv(res_fpath)
df

,Unnamed: 0,model,held out,feature set,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,params,...,split9_train_score,split10_train_score,split11_train_score,mean_train_score,std_train_score,param_activation,param_alpha,param_hidden_layer_sizes,param_learning_rate,param_solver
0,0,Random Forest,subject,"{'Min', 'Energy', 'MeanFreq', 'IQR', 'arCoeff'...",0.878352,0.027888,0.003233,0.000415,10.0,{'n_estimators': 10},...,0.965878,0.965555,0.968225,0.965971,0.001712,NaN,NaN,NaN,NaN,NaN
1,1,Random Forest,subject,"{'Min', 'Energy', 'MeanFreq', 'IQR', 'arCoeff'...",1.804766,0.065974,0.005504,0.000771,20.0,{'n_estimators': 20},...,0.980326,0.982197,0.982521,0.981154,0.000878,NaN,NaN,NaN,NaN,NaN
2,2,Random Forest,subject,"{'Min', 'Energy', 'MeanFreq', 'IQR', 'arCoeff'...",4.647198,0.199652,0.013108,0.002702,50.0,{'n_estimators': 50},...,0.988196,0.990066,0.989013,0.988590,0.000701,NaN,NaN,NaN,NaN,NaN
3,3,Random Forest,subject,"{'Min', 'Energy', 'MeanFreq', 'IQR', 'arCoeff'...",9.671321,0.739652,0.024006,0.003322,100.0,{'n_estimators': 100},...,0.990582,0.992066,0.991385,0.990724,0.000733,NaN,NaN,NaN,NaN,NaN
4,0,Gradient Boosting,subject,"{'Min', 'Energy', 'MeanFreq', 'IQR', 'arCoeff'...",17.824586,0.337792,0.001995,0.000280,10.0,{'n_estimators': 10},...,0.542927,0.530478,0.540046,0.541370,0.009937,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,91,MLP,subject,"{'Min', 'Energy', 'MeanFreq', 'IQR', 'arCoeff'...",7.068870,1.437706,0.002646,0.000426,NaN,"{'activation': 'relu', 'alpha': 0.05, 'hidden_...",...,0.301168,0.207379,0.229852,0.271346,0.034374,relu,0.05,"(50,)",adaptive,adam
100,92,MLP,subject,"{'Min', 'Energy', 'MeanFreq', 'IQR', 'arCoeff'...",3.012101,0.527593,0.003294,0.000508,NaN,"{'activation': 'relu', 'alpha': 0.05, 'hidden_...",...,0.215249,0.215249,0.208378,0.208251,0.005259,relu,0.05,"(100,)",constant,sgd
101,93,MLP,subject,"{'Min', 'Energy', 'MeanFreq', 'IQR', 'arCoeff'...",8.229035,2.249828,0.003388,0.000507,NaN,"{'activation': 'relu', 'alpha': 0.05, 'hidden_...",...,0.280978,0.245630,0.299894,0.262453,0.027812,relu,0.05,"(100,)",constant,adam
102,94,MLP,subject,"{'Min', 'Energy', 'MeanFreq', 'IQR', 'arCoeff'...",10.277087,0.514650,0.003371,0.000524,NaN,"{'activation': 'relu', 'alpha': 0.05, 'hidden_...",...,0.215249,0.215249,0.208752,0.208299,0.005210,relu,0.05,"(100,)",adaptive,sgd


In [7]:
# find the best performing row for each model type
# and extract test scores

cols_of_interest = [f'split{i}_test_score' for i in range(12)]

model_to_scores = {}
for model in np.unique(df['model']):
    print(f"model = {model}")
    max_model_score = df[df['model'] == model]['mean_test_score'].max()
    res = df[(df['model'] == model) & (df['mean_test_score'] == max_model_score)]
    res = res[cols_of_interest]
    test_scores = res.to_numpy()
    model_to_scores[model] = test_scores
#     res = dict(res.iloc[0])


model = Gradient Boosting
model = MLP
model = Random Forest


Null hypothesis is that the mean score is at chance level. Namely, the probability of guessing the correct class at random or worse.
$$H_0: \mu \leq \mu_0 = 0.2$$

The alternative hypothesis is that it is better than chance:
$$H_1: \mu > \mu_0$$

I don't have a variance for my data, so I am forced to use the hypothesis test for unknown variance. As I am predicting one way, it is a one tailed test. The test statistic is therefore:
$$TS = \sqrt{n} \frac{\bar{X}- \mu_0}{S}$$
where $S$ is the sample standard deviation.


In [8]:
alpha = 0.05
import scipy.stats
mu0 = 0.2
for model, scores in model_to_scores.items():
    print(f"model: {model}")
    Xbar = np.mean(scores)
    S = np.std(scores, ddof=1)
    print(f"Xbar = {Xbar}, S = {S}")
    TS = np.sqrt(12)*(Xbar - mu0)/S
    print(f"TS = {TS}")
    t_11 =  scipy.stats.t.ppf(1-alpha, scores.size-1, loc=mu0, scale=S)
    print(f"Threshold mean score to reject H_0 is {t_11:.4f}, mean score is {TS:.4f}")
    if t_11 < TS:
        print(f"At {100*(1-alpha)}% significance level we can reject H_0")
    pval = 1 - scipy.stats.t.cdf(TS, scores.size-1, loc=mu0, scale=S )
    print(f"We can reject H0 at any significance level greater than {pval:.3g}")
    print(f"We can reject H0 at any confidence level less than {100*(1-pval):.1f}")
    print()

model: Gradient Boosting
Xbar = 0.22389534372491018, S = 0.06615838129856139
TS = 1.2511778125008797
Threshold mean score to reject H_0 is 0.3188, mean score is 1.2512
At 95.0% significance level we can reject H_0
We can reject H0 at any significance level greater than 3.1e-09
We can reject H0 at any confidence level less than 100.0

model: MLP
Xbar = 0.2656577634163771, S = 0.10446720208528115
TS = 2.1771920732722503
Threshold mean score to reject H_0 is 0.3876, mean score is 2.1772
At 95.0% significance level we can reject H_0
We can reject H0 at any significance level greater than 4.83e-10
We can reject H0 at any confidence level less than 100.0

model: Random Forest
Xbar = 0.2141562653502543, S = 0.07961989149699139
TS = 0.6159106818926974
Threshold mean score to reject H_0 is 0.3430, mean score is 0.6159
At 95.0% significance level we can reject H_0
We can reject H0 at any significance level greater than 0.000142
We can reject H0 at any confidence level less than 100.0



In [10]:
list(range(0,100,10))

[0, 10, 20, 30, 40, 50, 60, 70, 80, 90]

## Tests to develop

| Test | Motivation |
| :---: |  :---: | 
| Hold one subject out balanced | Can test that there is some information for prediction between partitipants |
| Hold one subject-condition out (balanced) | Can test that there is some information for prediction within partitipants with unseen conditions |
| Hold one subject-condition-phase out (balanced) | Can test that there is some information for prediction within partitipant-condition |
